In [1]:
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px
import yfinance as yf
import dash_bootstrap_components as dbc
import geopandas as gpd
from urllib.request import urlopen
import json

In [2]:
# Load data
emissions = pd.read_csv('co2_emissions_kt_by_country.csv')
region = pd.read_csv('Methane_final.csv')
gdp = pd.read_csv('world_country_gdp_usd.csv')
pop = pd.read_csv('population_by_country_2020.csv')

region = region.drop(columns=['Unnamed: 0'])

gdp.columns = gdp.columns.str.replace(' ', '_')
gdp.columns = gdp.columns.str.lower()

pop = pop.rename(columns={'Country (or dependency)': 'country', 'Population (2020)': 'population', 'Density (P/Km²)': 'density', 
                          'Land Area (Km²)': 'land_area', 'Migrants (net)': 'migrants', 'Fert. Rate': 'fert_rate', 'Med. Age': 'med_age', 'Urban Pop %': 'urban_pop'})

In [3]:
with urlopen('https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson') as response:
    geo = json.load(response)
geo = gpd.GeoDataFrame.from_features(geo['features'])

In [4]:
geo = geo.rename(columns={'ADMIN': 'country'})

In [5]:
# merge emissions and gdp
merge = pd.merge(emissions, gdp, left_on='country_code', right_on='country_code', how='inner')
merge = merge.drop(columns=['country_name_y', 'year_y'])
merge = merge.rename(columns={'country_name_x': 'country_name', 'year_x': 'year'})
sec_merge = pd.merge(merge, region, left_on='country_name', right_on='country', how='inner')
final_merge = pd.merge(sec_merge, pop, left_on='country_name', right_on='country', how='inner')
df = final_merge.copy()

In [6]:
df = df[df['year'] >= 2000]

In [7]:
df = df.drop_duplicates(subset=['country_name', 'year'])

In [8]:
df = df.drop(columns=['country_x','country_y' , 'type', 'segment', 'reason', 'baseYear', 'notes'])
df = df.rename(columns={'country_name': 'country', 'value': 'co2_emissions', 'gdp_usd': 'gdp', 'gdp_per_capita_usd':'gdp_per_capita', 'emissions':'methane_emissions'})

df_2018 = df[df['year'] == 2018]

In [9]:
hist_data = df_2018[df_2018['co2_emissions'] > 200000]
px.histogram(hist_data, x='country', y='gdp_per_capita', color='region', title='GDP per capita per country')

In [10]:
px.histogram(hist_data, x='country', y='co2_emissions', color='region', title='CO2 emissions per country')

In [11]:
px.histogram(hist_data, x='country', y='methane_emissions', color='region', title='Methane emissions per country')

In [12]:
px.set_mapbox_access_token('pk.eyJ1IjoieW9ycmlkIiwiYSI6ImNsZnV3cHllejAxazUzZ2x5Y25ua29jbnIifQ.DaEnA0YRKUq6q5eNbj-o0g')

In [13]:
fig = px.choropleth_mapbox(df_2018, geojson=geo, featureidkey = 'properties.ISO_A3',
                           locations='country_code', color='co2_emissions',
                           color_continuous_scale="Viridis",
                           height=600, width=1000, center={"lat":48.661256, "lon":57.916474}, zoom=1, range_color=(0, 5000000),
                          )
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

KeyboardInterrupt: 

In [ ]:
df_show = df.head(100)

In [ ]:
df_show